In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import random
import os
import pickle
from tqdm import tqdm

from scipy.sparse import csr_matrix, save_npz
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

from app.source import *
from utils import draw_barplor
from app.recomendations import make_recomendations_with_cf \
                               , make_recomendations_with_genre \
                               , make_recomendations_with_sypnopsis \
                               , make_recomendations_with_genres_and_sypnopsis \
                               , make_recomendations_with_clustering \
                               , vectorization, get_scores

## Предобработка данных

### работа с признаками

In [ ]:
anime_ratings = pd.read_csv(ANIME_DIR + "animelist.csv", nrows=10000000)
anime_data = pd.read_csv(ANIME_DIR + "anime.csv")

Рассиморим, какая информация находится в файле $anime.csv$

In [ ]:
anime_data.info()

Сразу переименуем колонку $MAL\_ID$ в $anime\_id$

In [ ]:
anime_data.rename(columns={'MAL_ID':"anime_id"},inplace=True)
anime_data.columns

Избавимся от лишних полей

In [ ]:
to_keep = ['anime_id', 'Name', 'Score', 'Genres', 'Members']
anime_data = anime_data[to_keep]
anime_data

Сейчас у нас все жанры описаны в одном поле через запятую, что не очень удобно, поэтому мы определим все жанры и добавим их как поля для каждой записи

In [ ]:
genres_column = anime_data["Genres"].map(lambda x: x.split(", "))
genres = list(set(sum(genres_column, [])))

anime_data[genres] = 0
for i in range(0, len(genres_column)):
    anime_data.loc[i, genres_column[i]] = 1

anime_data = anime_data.drop(columns="Genres")

Заменим Unknown в поле Score на 0

In [ ]:
dict = {'Unknown' : 0}
anime_data['Score'] = anime_data['Score'].astype(str).apply(lambda x : dict[x] if x == 'Unknown' else x).astype(float)

anime_data.info()

Рассмотрим теперь информацию файла $animelist.csv$

In [ ]:
anime_ratings.info()

Избавимся от информации о кол-ве просмотренных эпизодов и статусе просмотра

In [ ]:
anime_ratings = anime_ratings[['user_id', 'anime_id', 'rating']]
anime_ratings.info()

Проверим, есть ли оценки для всех аниме, представленных в датасете

In [ ]:
anime_ratings.anime_id.nunique()

Это действительно так

Объеденим информацию из двух файлов

In [ ]:
anime_complete = pd.merge(anime_data, anime_ratings, on='anime_id')
anime_complete.info()

Переименуеем Score в total_score, а rating в user_score

In [ ]:
anime_complete = anime_complete.rename(columns={'Score' : 'total_score', 'rating': 'user_score'})

anime_complete.isna().sum()

сохраним полученый df в csv

In [ ]:
anime_complete.to_csv(ANIME_DIR + 'complete.csv')

### Подготовка данных для рекомендаций

In [ ]:
anime_feature = pd.read_csv(ANIME_DIR + 'complete.csv')
anime_feature = anime_feature.drop(columns='Unnamed: 0')
anime_feature.head()

Ради интереса посмотрим на 10 самых популярных аниме по кол-ву оценок и по кол-ву фанатов

In [ ]:
top10_by_score = anime_feature['Name'].value_counts().nlargest(10)
top10_by_members = anime_feature.sort_values(by='Members', ascending=False).drop_duplicates(subset='Name').head(10)

In [ ]:
draw_barplor(top10_by_score.index, top10_by_score.values, 
             "топ 10 по суммарному рейтингу", "название аниме", "суммарный рейтинг")

И тут в поезде анимешников завязалась драка...

In [ ]:
draw_barplor(top10_by_members['Name'], top10_by_members['Members'],
             "топ 10 по числу фанатов", "название аниме", "фанаты")

...в которую ворвались адепты Всемогущего, Сайтамы и 1000 - 7 ...

Проверим сколько в среднем поставил оценок каждый пользователь

In [ ]:
count_of_users = anime_feature['user_id'].value_counts()
count_of_users.describe()

Половина пользователей, оцекни которых мы собираемся использовать поставили
оценки меньше чем $67$ анииме, однако средним значением для выборки является около
$101$-ой оценки. Если рассматривать, что большая часть оценок поставлена по
просмотру аниме и при этом не для каждого просмотренного аниме пользователь
поставил оценку, то для дальнейшей работы
стоит выбрать пользователей, которые поставили оцеку $75$ и более аниме. 

Данное значение было взято из следующего:
- Мной просмотрено около 300 аниме, но при этом оценка выставлена лишь половине.
- Большая часть выходящих аниме преставляют собой 12-ти серийные сериалы,
  средняя продолжительность которых составляет $12 * 24 / 60 = 4.8$ часа.
- Следовательно человек, просмотревший $75$ аниме, потратил на это $360$ часов...

$360$ часов можно интерпретировать примерно как год работы кинокритика, т.к
кроме самого просмотра, человек ещё тратит какое-то время на осмысление сюжета,
понимание мотивов героев. Если он конечно смотрит их не залпом по несколько
аниме в день из-за кошкодевочек и им подобным)

In [ ]:
print(f'{type(count_of_users)}\n{count_of_users}')

In [ ]:
anime_feature = anime_feature[anime_feature['user_id'].isin(count_of_users[count_of_users >= 75].index)]
anime_feature.user_id.nunique()

создадим теперь таблицу, в которой строками будут названия аниме, а столбцами id пользователей. Значениями будут оценки

In [ ]:
anime_pivot = anime_feature.pivot_table(index='Name', columns='user_id',
                                        values='user_score').fillna(0)
anime_pivot.head()

также сохраним его для дальнейшей работы, предварительно преобразовав в csr_matrix для удобства разворачивания в дальнейшем

In [ ]:
cf_matrix = csr_matrix(anime_pivot.values)
save_npz(ANIME_DIR + 'cf_matrix.npz', cf_matrix)

## Коллаборативная рекомендация

In [ ]:
# cf_matrix = load_npz(ANIME_DIR + 'cf_matrix.npz')
anime_pivot = pd.read_csv(ANIME_DIR + 'anime_cf.csv', index_col='Name')
anime_pivot.head()

In [ ]:
anime_pivot.info()

Слишком много места требуется для данных. Выберем 4000 случайных пользователей и их оценки.

In [ ]:
copy_pivot = anime_pivot[random.sample(anime_pivot.columns.to_list(), 4000)]
copy_pivot.info()

In [ ]:
cf_matrix = csr_matrix(copy_pivot.values)

In [ ]:
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(cf_matrix)

In [ ]:
anime_titles = copy_pivot.index
anime_title = np.random.choice(anime_titles)
query_index = copy_pivot.index.get_loc(anime_title)
print(f"Randomly selected anime title: {anime_title} \n")

In [ ]:
distances, index = knn_model.kneighbors(copy_pivot.iloc[query_index, :]
                                        .values.reshape(1, -1)
                                        , n_neighbors=10)

print(f"Recommendations for {anime_pivot.index[query_index]}:\n")
for i, ind in enumerate(index.flatten()):
    print(f'''{i + 1}: {copy_pivot.index[ind]}, with distance {distances.flatten()[i]:{4}.{3}}''')

In [ ]:
result = make_recomendations_with_cf(copy_pivot, count_recomendations=20, model_path= 'models/cf_model.sav')

if result != None:
    print(f' {20} title recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with distance {value[1]:{4}.{3}}')

In [ ]:
result = make_recomendations_with_cf(copy_pivot, count_recomendations=20, model_path= 'cf_model.sav', name='aboba')

if result != None:
    print(f' {20} title recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with distance {value[1]:{4}.{3}}')
else:
    print('Anime not found')

## Рекомендации на основе жанра, описания

У метода коллаборативной фильтрации есть несколько минусов:
1. Объем данных
2. Существуют такие аниме, для которых есть слишком мало оценок, поэтому
   рекомендации выстраиваются на основе предпочтений тех людей, которые
   просмотрели это аниме и это не дает нам реальной пользы, т.к рекомендации
   начинают работать на основе вкуса 1-2 людей

In [ ]:
synopsis_data = pd.read_csv(ANIME_DIR + 'anime_with_synopsis.csv')
synopsis_data.rename(columns={'MAL_ID':"anime_id"},inplace=True)
synopsis_data

In [ ]:
print(synopsis_data.info())

634 KB звучит гораздо лучше, чем 500 MB при условии, что это вес используемого среза оценок по пользователям

In [ ]:
synopsis_data['Genres'] = synopsis_data['Genres'].fillna('')
genres = synopsis_data['Genres'].str.split(', ').astype(str)

In [ ]:
tfidfv = TfidfVectorizer()
tfidf_genres = tfidfv.fit_transform(genres)

In [ ]:
cos_sim = cosine_similarity(tfidf_genres, tfidf_genres)
print(type(cos_sim))

In [ ]:
anime_indexes = pd.Series(synopsis_data.index, index=synopsis_data['Name'])
print(anime_indexes)

In [ ]:
title = np.random.choice(anime_indexes.index)
print(title)

In [ ]:
cos_scores = sorted(list(enumerate(cos_sim[anime_indexes[title]]))
                    , key= lambda x: x[1]
                    , reverse=True)
recomendations = [i[0] for i in cos_scores[0:10]]
similarity = [i[1] for i in cos_scores[0:10]]

print(f'Recomendations for {title}:') 

for i, value in enumerate(recomendations):
        print(f'''{i + 1}: {synopsis_data["Name"]
                            .iloc[value]}, with similarity {similarity[i]:{4}.{3}}''')
    

In [ ]:
result = make_recomendations_with_genre(synopsis_data, name=title)

if result != None:
    print(f' Recomendations for {result[0]}')
    
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[3]:{4}.{3}}')
else:
    print('Anime not found')

In [ ]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Shingeki no Kyojin'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[3]:{4}.{3}}')
else:
    print('Anime not found')

... А что если проверить что-то, у чего нет 10+ частей?

In [ ]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Death Note'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Однако все равно сходство на основе жанра не может нам гарантировать то, что
рекомендоваться будет то, что действительно соответствует желаниям пользователя.

In [ ]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity in genre {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Получилось не совсем то, что планировалось, но всё же. Для здоровых людей:
- Аниме(конкретно для 12-ти серийника) для которого требовалось найти похожих
  --- представляет собой школьный ромком с небольшим количеством
  сверхестественного.
- Первой рекомендацией получили аниме, название которого само говорит за себя:
  <<Сквозь слезы я притворяюсь кошкой>>.

Проверим теперь рекомендации, основываясь на описании к тайтлу.

In [ ]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Seishun Buta Yarou wa Bunny Girl Senpai no Yume wo Minai'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Вроде, по моему личному опыту, это выглядит немного интереснее и более близко к тематике первоначального аниме.

In [ ]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Death Note'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Рассмотрим на более популярном аниме...

In [ ]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Мы получили 0 аниме, похожих хоть как-то на SAO(не считая его частей)...

In [ ]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

Это выглядит гораздо лучше:
- Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara ---
  очень похоже идейно(за исключением попаданства) на SAO и связано с игрой в vr.
- Btooom! --- попаданец в игру...
- Log Horizon --- попаданцы в игру + гг обоих аниме имеют какую-то общую
  особенность + чуточку романтикии...

In [ ]:
result = make_recomendations_with_genres_and_sypnopsis(synopsis_data
                                                       , name='Sword Art Online'.lower()
                                                       , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

При совместном использовании мы смогли увидеть Overlord и Аватар Короля, тайтлы очень похожие на SAO

## Подготовка изображений

In [ ]:
images = pd.read_csv( ANIME_DIR + 'anime_images.csv')
images = images[['title','images']]
images.head()

In [ ]:
ex = images['images'][0]
print(ex)

In [ ]:
images['image_url'] = images['images'].str.split( ' \'').apply(lambda x: x[1]).str.split(',').apply(lambda x: x[0].replace('\'', ''))
images = images[['title', 'image_url']]
images.head()

In [ ]:
images.to_csv('app/static/images_links.csv')

In [ ]:
print(result)

In [ ]:
img = [images[images['title'] == x[0]]['image_url'].values for x in result[1]]

In [ ]:
print(img[1])

In [ ]:
images[images['title'] == 'Sword Art Online: Progressive Movie - Hoshi Naki Yoru no Aria']['image_url'].values

## Кластеризация

In [ ]:
data = pd.read_csv(ANIME_DIR + 'anime_with_synopsis.csv')
data.info()

In [ ]:
vectorize_data = TfidfVectorizer().fit_transform(data['sypnopsis'].str.strip(',.!?:"()').str.split(' ').astype(str))

In [ ]:
nums_claster = list(map(int, range(1, 100)))
inertia = []

for i in tqdm(nums_claster):
    kmeans = KMeans(n_clusters= i, random_state=69, n_init=5)
    kmeans.fit(vectorize_data)
    inertia.append(kmeans.inertia_)



In [ ]:
sns.lineplot(inertia)
plt.xlabel('Count of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
vectorize_data = TfidfVectorizer().fit_transform(data['Genres'].str.split(', ').astype(str))
nums_claster = list(map(int, range(1, 100)))
inertia = []

for i in tqdm(nums_claster):
    kmeans = KMeans(n_clusters= i, random_state=69, n_init=5)
    kmeans.fit(vectorize_data)
    inertia.append(kmeans.inertia_)

In [ ]:
sns.lineplot(inertia)
plt.xlabel('Count of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
OPTIMUM_NUM_CLUSTER = 20
kmeans_model = KMeans(n_clusters= OPTIMUM_NUM_CLUSTER, random_state=69, n_init=15)
res_kmeans = kmeans_model.fit_predict(vectorize_data)
if not os.path.exists(CBF_CLUSTER_MODEL):
     pickle.dump(kmeans_model, open(CBF_CLUSTER_MODEL, 'wb'))
plt.hist(
    res_kmeans,
    bins=OPTIMUM_NUM_CLUSTER,
)

plt.title('Distribution by cluster')
plt.show()

In [ ]:
data.loc[:, 'cluster'] = res_kmeans

In [ ]:
title = np.random.choice(data.index)
print(data.iloc[title]['Name'])
num_cluster = data.iloc[title]['cluster']
print(num_cluster)
cluster_data = data[data['cluster'] == num_cluster]

In [ ]:
cluster_data.info()
cluster_data = cluster_data.reset_index()

In [ ]:
synopsis = cluster_data['sypnopsis'].str.strip(',.!?:"()') \
                                              .str.split(' ') \
                                              .astype(str)
similarity_matrix = vectorization(synopsis, CBF_SYPNOPSIS_DATA, cosine_similarity)

anime_indexes = pd.Series(cluster_data.index
                              , index=cluster_data['Name'])
name = data.iloc[title]['Name']
print(name)
similarity_scores = get_scores(similarity_matrix, anime_indexes, name)

ind = similarity_scores[0: 10]
recomendations = []

for _, index in enumerate(ind):
    title = cluster_data[['Name', 'Genres', 'sypnopsis']].iloc[index[0]] \
                                                        .tolist()
    title.append(index[1])
    recomendations.append(title)

print(f' Recomendations for {name}')
for i, value in enumerate(recomendations):
    print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')

## Сравнение

In [ ]:
synopsis_data = pd.read_csv(ANIME_DIR + 'anime_with_synopsis.csv')
vectorize_data = TfidfVectorizer().fit_transform(synopsis_data['Genres']
                                                     .str.split(', ')
                                                     .astype(str))
if os.path.exists(CBF_CLUSTER_MODEL):
     model = pickle.load(open(CBF_CLUSTER_MODEL, 'rb'))
else:
    
    model = KMeans(n_clusters= OPTIMUM_NUM_CLUSTER, random_state= 69
                                                  , n_init= 15)
    model.fit(vectorize_data)

res_model = model.predict(vectorize_data)

synopsis_data.loc[:, 'cluster'] = res_model

synopsis_data

In [ ]:
result = make_recomendations_with_genre(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

In [ ]:
result = make_recomendations_with_sypnopsis(synopsis_data
                                        , name='Sword Art Online'
                                        , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

In [ ]:
result = make_recomendations_with_genres_and_sypnopsis(synopsis_data
                                                       , name='Sword Art Online'
                                                       , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')

In [ ]:
result = make_recomendations_with_clustering(synopsis_data
                                             , name='Sword Art Online'
                                             , count_recomendations= 20)

if result != None:
    print(f' Recomendations for {result[0]}')
    for i, value in enumerate(result[1]):
        print(f'{i + 1}: {value[0]}, with similarity {value[3]:{4}.{3}}')
else:
    print('Anime not found')